# Load libraries

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce 

# Load data

In [2]:
# Load pickle file with the categorical features selected
import pickle
with open("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/final_selected_features.pkl", 'rb') as f:
    df_importance_150 = pickle.load(f)
    
list_features = list(df_importance_150.feature_name)

In [3]:
dat_cols = list(pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/train_date.csv", nrows=0, dtype=np.float32).columns)
num_cols = list(pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/train_numeric.csv", nrows=0, dtype=np.float32).columns)
cat_cols = list(pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/train_categorical.csv", nrows=0).columns)

# Python program to find the common elements  
# in two lists 
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b) 
  
    if (a_set & b_set): 
        lista = list(a_set & b_set) 
    else: 
        print("No common elements")  
    return lista
        
sel_cat = common_member(list_features, cat_cols)
sel_num = common_member(list_features, num_cols)
sel_dat = common_member(list_features, dat_cols)


In [17]:
date_chunks = pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/train_date.csv", usecols = sel_dat, chunksize=100000, dtype=np.float32)
num_chunks = pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/train_numeric.csv", chunksize=100000, dtype=np.float32, usecols = sel_num)
cat_chunks = pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/train_categorical.csv", usecols = sel_cat, chunksize=100000)
X = pd.concat([pd.concat([dchunk, nchunk, cchunk], axis=1)
               for dchunk, nchunk, cchunk in zip(date_chunks, num_chunks, cat_chunks)])

C:\Users\luisgasco\.conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (1987) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## Transform the categorical variable

In [20]:
import category_encoders as ce   # version 1.2.8
filename = "C://Users/luisgasco/Documents/bosh_kaggle_comp/data/encoder_for_testing.sav"
ce_target = pickle.load(open(filename, 'rb'))
ce_target

LeaveOneOutEncoder(cols=['L3_S32_F3854'], sigma=0.1, verbose=1)

In [21]:
X_trans = ce_target.transform(X)

# Load prediction model

In [22]:
# open a file, where you ant to store the data
filename = "C://Users/luisgasco/Documents/bosh_kaggle_comp/data/final_prediction_model.sav"
model_fin = pickle.load(open(filename, 'rb'))


In [23]:
# Make predictions with the threshold 0.246
predicciones_final = (model_fin.predict_proba(X_trans)[:,1] > 0.246).astype(np.int8)

# Save to the submission csv

In [30]:
sub = pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/sample_submission.csv", index_col=0)

C:\Users\luisgasco\.conda\envs\tensorflow\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [33]:
sub.shape

(1183748, 1)

In [34]:
sub["Response"] = predicciones_final

ValueError: Length of values does not match length of index

In [35]:
sub.to_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/submission.csv")